# tree functions

Goal 
  take a graph model, and pull a subtree out of it
    


In [1]:
class Person():
    def __init__(self, name, gender, birth, death='-', religion='A', parent=None):
        self.name = name
        self.gender = gender
        self.birth = birth
        self.death = death
        self.religion = religion
        self.parent = parent   # used only for loading data

    def weight(self):
        """ this is used to sort the siblings for succession """
        if self.gender == 'F':
            return 2000 + int(self.birth)
        return int(self.birth)
        
        
    def __repr__(self):
        return '{} ({}, {},{},{})'.format(self.name, self.gender, self.birth, self.death, self.religion)

    
def find_root(family):
    """ finds the element in the people list for root """
    return [x for x in family if x.parent == '-'][0]

    
    
class FamilyTree():
    def __init__(self, root=None, mapping=None):
        self.root = root
        self.mapping = mapping if mapping else {}

        
    
        
    def __repr__(self):
        return 'Root "{}" mapping: {}'.format(self.root, self.mapping)

    def _child_to_add(self, family):
        """
        finds a person in the family that can be added to the current tree
        """
        for person in family:
            # if root or already had children
            if person.parent == self.root.name \
               or person.parent in self.mapping:
                return person
            # look in all the values for potential new child
            for _, kids in self.mapping.items():
                for kid in kids:
                    if person.parent == kid.name:
                        return person
            #print('no match {}'.format(person), file=sys.stderr)
        return None

    def load_tree(self, family):
        """
        loads a tree with a list of Person records
        """
        self.root = find_root(family)
        family.pop(family.index(self.root))

        while family:
            child = self._child_to_add(family)
            if child is None:
                break
            if child.parent in self.mapping:
                self.mapping[child.parent].append(child)
            else:
                self.mapping[child.parent] = [ child ]
            #print('added {} to mapping'.format(child), file=sys.stderr)
            family.pop(family.index(child))
            
            
        # go through and order the children maps
        for parent, children in self.mapping.items():
            children.sort(key=lambda x: x.weight())
    
    
    
    def subtree(self, node):
        """ returns the subtree from this element """
        tree = {node.name: self.mapping.get(node.name, []).copy()}
        for n in self.descendents(node.name):
            if self.mapping.get(n.name):
                tree[n.name] = self.mapping[n.name].copy()
        return FamilyTree(node, tree)
                        
        
    def descendents(self, node):
        """ returns a list of all descendents from this node """
        all_kids = self.mapping.get(node, []).copy()
        for elem in all_kids.copy():
            all_kids += self.descendents(elem.name)
        return all_kids

    def succession(self):
        """
        starts with the root (unless dead)
        then calls succession on each child in order
        """
        print('succession:{}'.format(self))
        s = [ self.root.name ] if self.root.death == '-' else []
        for child in self.mapping[ self.root.name ]:
            s += self.subtree(child).succession()
        print('return:{}'.format(s))
        return s
    
    
    
tree_map = {
    'Elizabeth': [ Person('Charles', 'M', 1946) ],
    'Charles' : [ Person('William', 'M', 1975, death=2018), Person('Harry', 'M', 1978) ],
    'William' : [ Person('George', 'M', 2000),]    
}
f = FamilyTree(Person('Elizabeth', 'F', 1910), tree_map)
print(f.descendents('Charles'))
print(f.descendents('William'))
print(f.subtree(Person('Charles', 'M', 1975)))
# e = Person('Elizabeth', 1910, '-')
# c = Person('Elizabeth', 1946, '-')
# w = Person('Elizabeth', 1975, '-')
# h = Person('Elizabeth', 1978, '-')
# g = Person('Elizabeth', 1910, '-')
# f = FamilyTree('a', {'a': ['b', 'c'], 'b': ['e'], 'e': ['f', 'g'], 'c': ['d'], 'g': ['h']})
print(f)
# print(f.subtree('b'))
# print(f.subtree('c'))

This tree should have succession as:
    Elizabeth, Charles, George, Harry  (assume William died)
    
    

In [2]:
print(f.succession())




In [3]:
raw = """KingGeorgeVI - 1895 1952 M Anglican
QueenElizabethII KingGeorgeVI 1926 - F Anglican
CharlesPrinceofWales QueenElizabethII 1948 - M Anglican
PrinceWilliamDukeofCambridge CharlesPrinceofWales 1982 - M Anglican
PrinceGeorgeofCambridge PrinceWilliamDukeofCambridge 2013 - M Anglican
PrincessCharlotteofCambridge PrinceWilliamDukeofCambridge 2015 - F Anglican
PrinceHenryofWales CharlesPrinceofWales 1984 - M Anglican
PrinceAndrewDukeofYork QueenElizabethII 1960 - M Anglican
PrincessBeatriceofYork PrinceAndrewDukeofYork 1988 - F Anglican
PrincessEugenieofYork PrinceAndrewDukeofYork 1990 - F Anglican
PrinceEdwardEarlofWessex QueenElizabethII 1964 - M Anglican
JamesViscountSevern PrinceEdwardEarlofWessex 2007 - M Anglican
LadyLouiseWindsor PrinceEdwardEarlofWessex 2003 - F Anglican
AnnePrincessRoyal QueenElizabethII 1950 - F Anglican
PeterPhillips AnnePrincessRoyal 1977 - M Anglican
SavannahPhillips PeterPhillips 2010 - F Anglican
IslaPhillips PeterPhillips 2012 - F Anglican
ZaraTindall AnnePrincessRoyal 1981 - F Anglican
MiaTindall ZaraTindall 2014 - F Anglican
PrincessMargaretCountessofSnowdon KingGeorgeVI 1930 2002 F Anglican
DavidArmstrong-Jones2ndEarlofSnowdon PrincessMargaretCountessofSnowdon 1961 - M Anglican
CharlesArmstrong-JonesViscountLinley DavidArmstrong-Jones2ndEarlofSnowdon 1999 - M Anglican
LadyMargaritaArmstrong-Jones DavidArmstrong-Jones2ndEarlofSnowdon 2002 - F Anglican
LadySarahChatto PrincessMargaretCountessofSnowdon 1964 - F Anglican
SamuelChatto LadySarahChatto 1996 - M Anglican
ArthurChatto LadySarahChatto 1999 - M Anglican"""

people = [Person(s[0], s[4], s[2], death=s[3], religion=s[5], parent=s[1]) for s in [l.split(' ') for l in raw.split('\n')]]

print(people[0])
print(people[-1])
print(find_root(people))

In [4]:
f = FamilyTree()
f.load_tree(people)
print(f)

Root "KingGeorgeVI (M, 1895,1952,Anglican)" mapping: {'KingGeorgeVI': [QueenElizabethII (F, 1926,-,Anglican), PrincessMargaretCountessofSnowdon (F, 1930,2002,Anglican)], 'QueenElizabethII': [CharlesPrinceofWales (M, 1948,-,Anglican), PrinceAndrewDukeofYork (M, 1960,-,Anglican), PrinceEdwardEarlofWessex (M, 1964,-,Anglican), AnnePrincessRoyal (F, 1950,-,Anglican)], 'CharlesPrinceofWales': [PrinceWilliamDukeofCambridge (M, 1982,-,Anglican), PrinceHenryofWales (M, 1984,-,Anglican)], 'PrinceWilliamDukeofCambridge': [PrinceGeorgeofCambridge (M, 2013,-,Anglican), PrincessCharlotteofCambridge (F, 2015,-,Anglican)], 'PrinceAndrewDukeofYork': [PrincessBeatriceofYork (F, 1988,-,Anglican), PrincessEugenieofYork (F, 1990,-,Anglican)], 'PrinceEdwardEarlofWessex': [JamesViscountSevern (M, 2007,-,Anglican), LadyLouiseWindsor (F, 2003,-,Anglican)], 'AnnePrincessRoyal': [PeterPhillips (M, 1977,-,Anglican), ZaraTindall (F, 1981,-,Anglican)], 'PeterPhillips': [SavannahPhillips (F, 2010,-,Anglican), IslaP

In [5]:
print(f.succession())


succession:Root "KingGeorgeVI (M, 1895,1952,Anglican)" mapping: {'KingGeorgeVI': [QueenElizabethII (F, 1926,-,Anglican), PrincessMargaretCountessofSnowdon (F, 1930,2002,Anglican)], 'QueenElizabethII': [CharlesPrinceofWales (M, 1948,-,Anglican), PrinceAndrewDukeofYork (M, 1960,-,Anglican), PrinceEdwardEarlofWessex (M, 1964,-,Anglican), AnnePrincessRoyal (F, 1950,-,Anglican)], 'CharlesPrinceofWales': [PrinceWilliamDukeofCambridge (M, 1982,-,Anglican), PrinceHenryofWales (M, 1984,-,Anglican)], 'PrinceWilliamDukeofCambridge': [PrinceGeorgeofCambridge (M, 2013,-,Anglican), PrincessCharlotteofCambridge (F, 2015,-,Anglican)], 'PrinceAndrewDukeofYork': [PrincessBeatriceofYork (F, 1988,-,Anglican), PrincessEugenieofYork (F, 1990,-,Anglican)], 'PrinceEdwardEarlofWessex': [JamesViscountSevern (M, 2007,-,Anglican), LadyLouiseWindsor (F, 2003,-,Anglican)], 'AnnePrincessRoyal': [PeterPhillips (M, 1977,-,Anglican), ZaraTindall (F, 1981,-,Anglican)], 'PeterPhillips': [SavannahPhillips (F, 2010,-,Angli

In [6]:
#19
n = 2
s = list(map(int,'2 5 1 3 4 4 3 5 1 1 2 1 4 1 3 3 4 2 1'.split(' ')))
s2 = [sum(s[ndx:(ndx+n)]) for ndx in range(0, len(s)-n+1)]
#18 7

print(s)
print(s2)

[2, 5, 1, 3, 4, 4, 3, 5, 1, 1, 2, 1, 4, 1, 3, 3, 4, 2, 1]
[7, 6, 4, 7, 8, 7, 8, 6, 2, 3, 3, 5, 5, 4, 6, 7, 6, 3]
